# Load data and classifiers

In [1]:
from main import *
from sklearn.metrics import confusion_matrix
import eli5
import matplotlib.pyplot as plt
LOAD_SAVED = True
OVERWRITE_SAVED = False
train_X, train_Y, test_X, test_Y, summary_cv, review_cv = memo.get_data(LOAD_SAVED, OVERWRITE_SAVED, CATEGORIES, 0.75, 1000)
clf_nb = memo.get_classifier(LOAD_SAVED, OVERWRITE_SAVED, trainer.naive_bayes, train_X, train_Y, 'clf_nb')
clf_lr = memo.get_classifier(LOAD_SAVED, OVERWRITE_SAVED, trainer.logistic_regression, train_X, train_Y, 'clf_lr')
clf_svm = memo.get_classifier(LOAD_SAVED, OVERWRITE_SAVED, trainer.svm, train_X, train_Y, 'clf_svm')
summary_cv = memo.load_pkl('static/summary_cv')
review_cv = memo.load_pkl('static/review_cv')

ImportError: dlopen(/anaconda3/lib/python3.6/site-packages/matplotlib/ft2font.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /anaconda3/lib/libfreetype.6.dylib
  Reason: Incompatible library version: libfreetype.6.dylib requires version 51.0.0 or later, but libpng16.16.dylib provides version 49.0.0

# Visualize classifiers

## Weight skews (Logistic Regression)

In [2]:
feature_names = summary_cv.get_feature_names() + review_cv.get_feature_names() + ['helpfulness rating'] + ['overall rating'] + ['review time']

In [3]:
eli5.show_weights(clf_lr,top=5,feature_names=feature_names)

## Example Prediction

In [4]:
eli5.show_prediction(clf_lr, test_X.getrow(100), feature_names=feature_names)

Some notes about our features:
- Features like review time, helpfulness rating, and overall rating seemed to have negative contributions to almost all category predictions.
- Our bias was always a strong feature, however almost always negative.

# Predict

## Naive Bayes

In [10]:
test_prd_nb = clf_nb.predict(test_X)
test_err_nb = tester.error_ratio(test_Y, test_prd_nb)
cm = confusion_matrix(test_Y, test_prd_nb)
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

[[ 86  10   0   0   0   1   3   1   3   0   0   0   0   1  11   2   0   0
    0   0   0   0   0   0]
 [  0  78   0   0   0   1   0   4   3   0   7   0   0   0   2   0   1   0
    0   0   0   1   0   4]
 [  0   0  81   0   1   0   0   6   4   0   1   2   1   0   0   0   0   3
    8   0  11   1   0   0]
 [  0   1   1 103   1   0   1   5   3   0   0   1   3   0   0   0   1   0
    0   1   0   0   0   0]
 [  0   0   3   0  92   0   0   9   1   0   0   0   2   0   1   0   0   0
    3   1   0   0   0   0]
 [  1   3   0   0   0  78   1   3   1   0   1   0   1   0  21   0   0   0
    0   0   0   0   1   0]
 [  1   1   0   0   0   2  87   2   0   1   0   0   0   0   2  11   0   0
    0   1   0   0   0   0]
 [  0   3   2   1   3   0   0  92   1   0   5   0   3   0   1   0   3   0
    0   0   1   1   0   0]
 [  0   1   0   0   0   0   0   5  98   0   3   0   1   0   0   0   1   0
    1   0   6   0   0   0]
 [  0   4   0   0   0   1   0   0   2  95   0   0   0   0   3   2   2   0
    0   0   0   0

## Logistic Regression

In [6]:
test_prd_lr = clf_lr.predict(test_X)
test_err_lr = tester.error_ratio(test_Y, test_prd_lr)

## Ensemble

In [7]:
test_predictions = [test_prd_nb, test_prd_lr]
test_prd_ensemble = tester.predict_ensemble(test_X, test_predictions)
test_err_ensemble = tester.error_ratio(test_Y, test_prd_ensemble)

# Results

In [8]:
print("Naive Bayes Testing Error:         ",test_err_nb)
print("Logistic Regression Testing Error: ",test_err_lr)
print("Ensemble Testing Error:            ",test_err_ensemble)

Naive Bayes Testing Error:          0.24665461121157323
Logistic Regression Testing Error:  0.17902350813743217
Ensemble Testing Error:             0.24665461121157323
